In [122]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score

In [123]:
df = pd.read_csv('heart.csv')

# One-hot encode categorical variables
df = pd.get_dummies(df, drop_first=True)

# Features and target
X = df.drop(columns='HeartDisease')
y = df['HeartDisease'].values

# print(X)
# print(y)

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# print(X_scaled)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [124]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

In [125]:
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [126]:
import torch.nn as nn

class Model(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(32, 1),
            # nn.Sigmoid()
        )

    def forward(self, x):
        return self.network(x)

model = Model(X_train.shape[1])

In [127]:
# criterion = nn.BCELoss()
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
epochs = 1000

In [128]:
for epoch in range(epochs):

    model.train()
    train_loss = 0.0

    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)
    if (epoch+1) % 100 == 0:
        print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}")

Epoch 100, Train Loss: 0.3210
Epoch 200, Train Loss: 0.2981
Epoch 300, Train Loss: 0.2815
Epoch 400, Train Loss: 0.2682
Epoch 500, Train Loss: 0.2486
Epoch 600, Train Loss: 0.2408
Epoch 700, Train Loss: 0.2281
Epoch 800, Train Loss: 0.2192
Epoch 900, Train Loss: 0.2192
Epoch 1000, Train Loss: 0.2153


In [131]:
model.eval()
with torch.no_grad():
    logits = model(X_test_tensor)
    probabilities = torch.sigmoid(logits)  # apply sigmoid
    preds = (probabilities >= 0.5).float()

    acc = accuracy_score(y_test_tensor.numpy(), preds.numpy())
    roc = roc_auc_score(y_test_tensor.numpy(), probabilities.numpy())

    print(f"Test Accuracy: {acc:.4f}")
    print(f"ROC-AUC Score: {roc:.4f}")


Test Accuracy: 0.9185
ROC-AUC Score: 0.9515
